`Time.now`是一个间接输入。首先我们引用了`Time`类，然后把`#now`消息发送给`Time`。我们想要的其实是方法`now`的返回值，而不是`Time`常量本身。任何时候为了得到方法返回值，而向其他对象发送消息，其实就是在使用间接输入。

间接输入层次越深，代码的耦合程度就越高

In [1]:
Place = Struct.new(:index, :name, :prize)

Place

In [3]:
first = Place.new(0, "first", "Peasant's Quest game")

#<struct Place index=0, name="first", prize="Peasant's Quest game">

In [4]:
second = Place.new(1, "second", "Limozeen Album")

#<struct Place index=1, name="second", prize="Limozeen Album">

In [9]:
thrid = Place.new(2, "third", "butter-da")

#<struct Place index=2, name="third", prize="butter-da">

In [10]:
winners = ["a", "b"]

["a", "b"]

In [34]:
[first, second, thrid].each do |place|
  p "In #{place.name} place, #{winners[place.index]}"
  p "you wind: #{place.prize}"
end

"In first place, a"
"you wind: Peasant's Quest game"
"In second place, b"
"you wind: Limozeen Album"
"In third place, "
"you wind: butter-da"


[#<struct Place index=0, name="first", prize="Peasant's Quest game">, #<struct Place index=1, name="second", prize="Limozeen Album">, #<struct Place index=2, name="third", prize="butter-da">]

`#to_s`就是一个显式转换方法。显式转换一般用于这样的情形：源类型和目标类型很大程度上不相关或者毫无关联

与之对应，`#to_str`就是一个隐式类型转换方法。隐式类型转换适用于元类型和目标类型很相近的情形

In [15]:
class EmacsConfigFile
  def initialize
    @filename = "#{ENV['HOME']}/.vimrc"
  end
  
  def to_path
    @filename
  end
end

:to_path

In [16]:
emacs_config = EmacsConfigFile.new

#<EmacsConfigFile:0x00000055c47320e8 @filename="/home/demouser/.vimrc">

In [17]:
File.open(emacs_config).lines.count

<main>: warning: IO#lines is deprecated; use #each_line instead


740

这是因为，`EmacsConfig`类定义了转换方法`#to_path`，而`File#open`又会在参数对象上调用`#to_path`方法，以便得到文件名字符串。这样一来，这个非字符串对象也能工作得很好。

下面的例子可以说明两者（隐式转化和显示转化的区别）。`Time`对象并非`String`，有太多方式可以将`Time`转化成`String`了。另外，这两种类型还是不想管的，因此`Time`只预定义了显式的类型转换方法`#to_s`，而没有定义隐式版的`#to_str`

In [20]:
now = Time.now
now.respond_to?(:to_s)
now.to_s
now.respond_to?(:to_str)

false

In [21]:
"hello, " + "world"

"hello, world"

如果`#to_str`所做的一切就是返回字符串自身，那么它看起来意义不大。`#to_str`的意义在于：许多`Ruby`方法都期望得到字符串输入，它们隐式地在输入对象上调用`#to_str`方法（这正是“隐式类型转换”这个术语的来源）

In [22]:
"I am a String".to_str

"I am a String"

In [28]:
class ArticleTitle
  def initialize(text)
    @text = text
  end
  
  def slug
    @text.strip.tr_s("A-Za-z0-0", "-").downcase
  end
  
  def to_str
    @text
  end
  
  def to_s
    to_str
  end
end

:to_s

In [30]:
title = ArticleTitle.new("A Modest Proposal")
"Today's future" +title

"Today's futureA Modest Proposal"

你可能还记得我说过Ruby核心类从来不会自动调用显式类型转换方法，但仍有例外，有一种情况下，Ruby语言却会自动调用显式类型转换方法。

In [31]:
"The time is now: #{Time.now}"

"The time is now: 2019-01-07 15:21:59 +0000"

In [33]:
PHONE_EXTENSIONS=["Operator", "Sales", "Customer Service"]

def dial_extension(dialed_number)
  dialed_number = dialed_number.to_i
  extension = PHONE_EXTENSIONS[dialed_number]
  puts "Please hold while you are connected to #{extension}"
end

dial_extension(nil)

<main>: warning: already initialized constant PHONE_EXTENSIONS


Please hold while you are connected to Operator


如果只关心得到预期输入类型而不深究其源类型，则可以使用显式类型转化方法。

In [35]:
def set_centrifuge_speed(new_rpm)
  new_rpm = new_rpm.to_int
  puts "Adjusting centrifuge to #{new_rpm} RPM"
end

:set_centrifuge_speed

In [36]:
bad_input = nil
set_centrifuge_speed(bad_input)

NoMethodError: undefined method `to_int' for nil:NilClass

In [37]:
aaa = nil
"fe#{aaa}"

"fe"

某种情况下，如果方法的参数不符合预期类型或者与预期不接近，就会为程序带来隐患，这时最好用隐式类型转换

这里的隐式类型转换起到了“门卫”的作用：用以确保方法接受到的参数要么是`Integer`，要么可以很好地被转化为`Integer`。当参数不满足条件时，`NoMethodError`清楚地告诉了代码调用者调用非法。